### Практическое задание 3
1. Постройте нейронную сеть(берем простую линейную сеть, которую разбирали на уроке: меняем число слоев, число нейронов , типы активации, тип оптимизатора) на датасет from sklearn.datasets import load_boston.

2. Измените функцию потерь и метрику для этой задачи. Постройте 10-15 вариантов и сведите результаты их работы в таблицу Опишите, какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?

3. Поработайте с документацией TensorFlow 2. Найти 2-3 полезные команды TensorFlow, не разобранные на уроке (полезные для Вас).

1-2. (*) Попробуйте обучить нейронную сеть на TensorFlow 2 на датасете imdb_reviews .Опишите, какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?

In [1]:
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin

from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [2]:
X, y = load_boston(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=42)

train_df = pd.DataFrame(X_train, columns=load_boston().feature_names)
test_df = pd.DataFrame(X_test, columns=load_boston().feature_names)

In [3]:
train_df.head(3)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21
1,0.13158,0.0,10.01,0.0,0.547,6.176,72.5,2.7301,6.0,432.0,17.8,393.30,12.04
2,0.17142,0.0,6.91,0.0,0.448,5.682,33.8,5.1004,3.0,233.0,17.9,396.90,10.21


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354 entries, 0 to 353
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     354 non-null    float64
 1   ZN       354 non-null    float64
 2   INDUS    354 non-null    float64
 3   CHAS     354 non-null    float64
 4   NOX      354 non-null    float64
 5   RM       354 non-null    float64
 6   AGE      354 non-null    float64
 7   DIS      354 non-null    float64
 8   RAD      354 non-null    float64
 9   TAX      354 non-null    float64
 10  PTRATIO  354 non-null    float64
 11  B        354 non-null    float64
 12  LSTAT    354 non-null    float64
dtypes: float64(13)
memory usage: 36.1 KB


In [5]:
train_df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
count,354.000000,354.000000,354.000000,354.000000,354.000000,354.000000,354.000000,354.000000,354.000000,354.000000,354.000000,354.000000,354.000000
mean,3.469887,11.403955,11.133051,0.073446,0.557259,6.325672,68.799718,3.765874,9.437853,407.042373,18.277966,359.701808,12.421130
std,8.315831,22.608457,6.938651,0.261237,0.116792,0.719211,27.665360,2.126032,8.639971,166.522237,2.256792,86.924779,7.112402
min,0.009060,0.000000,1.210000,0.000000,0.385000,3.863000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000
25%,0.082320,0.000000,5.130000,0.000000,0.450000,5.889500,46.025000,2.073700,4.000000,279.000000,16.650000,376.782500,6.862500
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.227500,77.350000,3.107300,5.000000,330.000000,18.600000,391.600000,10.925000
75%,3.281688,20.000000,18.100000,0.000000,0.631000,6.681750,93.475000,5.400700,24.000000,666.000000,20.200000,396.172500,16.225000
max,88.976200,95.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000


In [6]:
class NumberSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]

In [7]:
cont_transformers = []
cont_columns = ['CRIM', 'ZN', 'INDUS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX','PTRATIO', 'B', 'LSTAT']

cat_transformers = []
cat_columns = ['CHAS']

for col in cont_columns:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=col)),
                ('standard', StandardScaler())
            ])
    cont_transformers.append((col, transfomer))

for col in cat_columns:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=col)),
            ])
    cat_transformers.append((col, transfomer))

feats = FeatureUnion(cont_transformers+cat_transformers)

In [8]:
X_train = feats.fit_transform(train_df)
X_test = feats.transform(test_df)

In [9]:
model = Sequential([
                    Dense(32, activation='linear', input_shape=(13,)),
                    Dense(16, activation='linear'),
                    Dense(1, activation='linear')
                    ])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                448       
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 993
Trainable params: 993
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(loss=tf.keras.losses.MeanSquaredError(),
              metrics=tfa.metrics.RSquare(dtype=tf.float32, y_shape=(1,)))

In [11]:
h = model.fit(X_train, y_train, epochs=30, validation_split=0.15, verbose=0)

test_loss, test_r2 = model.evaluate(X_test, y_test, verbose=0)

print(f'R2 train - {h.history.get("r_square")[-1]:.3f};\n'
      f'R2 val - {h.history.get("val_r_square")[-1]:.3f};\n'
      f'R2 test - {test_r2:3f}.')

R2 train - 0.624;
R2 val - 0.555;
R2 test - 0.573507.


In [12]:
%%time

tf.random.set_seed(42)

neuron_num_list  = [128, 256]
epochs_list = [100, 200]
lr_list = [1e-3, 1e-2]
activation_func_list = ['linear', 'relu']
answer_list = []

for neuron in neuron_num_list:
    for epoch in epochs_list:
        for lr in lr_list:
            for optim in [tf.keras.optimizers.SGD(learning_rate=lr),
                          tf.keras.optimizers.RMSprop(learning_rate=lr),
                          tf.keras.optimizers.Adam(learning_rate=lr),
                          tf.keras.optimizers.Nadam(learning_rate=lr)]:
                for activation_func in activation_func_list:
        
                    model = Sequential([Dense(neuron, activation=activation_func, input_shape=(13,)),
                                        Dense(int(neuron/2), activation=activation_func),
                                        Dense(1, activation=activation_func)])

                    model.compile(optimizer=optim,
                                loss=tf.keras.losses.MeanSquaredError(),
                                metrics=tfa.metrics.RSquare(dtype=tf.float32, y_shape=(1,)))
                    
                    h = model.fit(X_train, y_train, epochs=epoch, validation_split=0.15, verbose=0)
                    test_loss, test_r2 = model.evaluate(X_test, y_test, verbose=0)
                    
                    answer_list.append([f'{neuron}, {epoch}, {lr}, {type(optim).__name__}, {activation_func}',
                                       h.history.get("r_square")[-1],
                                       h.history.get("val_r_square")[-1],
                                       test_r2,
                                       ])

Wall time: 3min 45s


In [13]:
comparing_df = pd.DataFrame(answer_list, columns=['model_settings', 'R2 train', 'R2 val', 'R2 test'])
comparing_df.sort_values(['R2 test'], ascending=False)

,model_settings,R2 train,R2 val,R2 test
9,"128, 100, 0.01, SGD, relu",0.971888,0.865409,0.871812
43,"256, 100, 0.01, RMSprop, relu",0.907876,0.806647,0.871774
19,"128, 200, 0.001, RMSprop, relu",0.971339,0.820064,0.864258
11,"128, 100, 0.01, RMSprop, relu",0.931116,0.833797,0.863823
29,"128, 200, 0.01, Adam, relu",0.984299,0.827718,0.861932
...,...,...,...,...
63,"256, 200, 0.01, Nadam, relu",-5.780224,-8.020116,-6.150573
8,"128, 100, 0.01, SGD, linear",NaN,NaN,NaN
24,"128, 200, 0.01, SGD, linear",NaN,NaN,NaN
40,"256, 100, 0.01, SGD, linear",NaN,NaN,NaN


Метрики удалось улучшить примерно до следующих значений:
* R2 train - 0.971
* R2 val - 0.865
* R2 test - 0.872

Количество нейронов в первом внутреннем слое и Количество эпох не сильно влияли на изменение метрик. Так же как и оптимизаторы - метрики не сильно отличались при прменении разных оптимизаторов.

Наибольшее влияние в улучшение точности нейросети внесло изменение функции активации с linear на relu.

Конечно имеется достаточно сильное переобучение. Пока не соображу, как можно бороться с переобучением, используя уже пройденное по нейросетям.
